# Lab 4.3: Vanishing/Exploding Gradients with TensorFlow

**Duration**: 45 minutes

## Learning Objectives
By the end of this lab, you will be able to:
- Identify vanishing and exploding gradient problems using TensorFlow tools
- Use TensorFlow's built-in solutions (BatchNormalization, gradient clipping)
- Monitor gradient flow with TensorFlow callbacks and metrics
- Apply modern gradient stabilization techniques in production code
- Compare manual understanding with TensorFlow's automated solutions

## Prerequisites
- Completed Labs 4.1 and 4.2
- Understanding of gradient problems from manual implementation
- Basic TensorFlow/Keras knowledge

## Lab Overview
This lab demonstrates how TensorFlow provides built-in solutions for gradient problems we learned to diagnose and solve manually. You'll see how modern frameworks make gradient stabilization accessible while building on your foundational understanding.

## Part 1: Environment Setup and Imports

### Instructions:
1. Run this cell to import all necessary libraries
2. Verify all imports are successful
3. If any imports fail, install missing packages using: `pip install numpy matplotlib scikit-learn`

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks, optimizers
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Configure matplotlib
plt.style.use('default')
plt.rcParams['figure.figsize'] = (10, 6)

print("🚀 TensorFlow Gradient Analysis Lab Ready!")
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")
print(f"GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")

print("\n" + "="*60)
print("MANUAL vs TENSORFLOW GRADIENT SOLUTIONS")
print("="*60)
print("""
What we learned manually:
📚 Gradient computation: dW = (1/m) * dZ * A_prev.T
📚 Problem diagnosis: Checking ||dW|| across layers
📚 Manual gradient clipping: scaling gradients by norm
📚 Batch norm implementation: (X - μ) / σ normalization

What TensorFlow provides:
✅ tf.keras.layers.BatchNormalization() - Optimized implementation
✅ tf.keras.utils.clip_gradients() - Built-in clipping
✅ tf.keras.callbacks for gradient monitoring
✅ Automatic gradient computation with GradientTape
✅ Initialization strategies in layer constructors

Key insight: Understanding the manual implementation helps you:
💡 Debug when TensorFlow solutions don't work as expected
💡 Customize gradient handling for specific use cases
💡 Understand what's happening "under the hood"
""")

## Part 2: Understanding Gradient Flow

### Instructions:
1. Run the following cells to create a deep network architecture
2. Observe how gradients change as they flow backward through layers
3. Identify patterns that lead to vanishing or exploding gradients

In [ ]:
class DeepNeuralNetwork:
    """Deep neural network with gradient tracking capabilities"""
    
    def __init__(self, layer_dims, activation='sigmoid', init_type='random'):
        """
        Initialize deep neural network
        
        Args:
            layer_dims: List of layer dimensions [input, hidden1, hidden2, ..., output]
            activation: Activation function type ('sigmoid', 'tanh', 'relu')
            init_type: Weight initialization type ('random', 'xavier', 'he')
        """
        self.layer_dims = layer_dims
        self.num_layers = len(layer_dims) - 1
        self.activation = activation
        self.init_type = init_type
        
        # Initialize parameters
        self.parameters = self._initialize_parameters()
        
        # Storage for gradient analysis
        self.gradient_norms = {}
        self.activation_stats = {}
        self.cache = {}
        
    def _initialize_parameters(self):
        """Initialize network parameters based on initialization type"""
        parameters = {}
        
        for l in range(1, self.num_layers + 1):
            n_prev = self.layer_dims[l-1]
            n_curr = self.layer_dims[l]
            
            if self.init_type == 'random':
                # Random initialization (prone to gradient problems)
                parameters[f'W{l}'] = np.random.randn(n_curr, n_prev) * 0.01
            elif self.init_type == 'xavier':
                # Xavier initialization (good for sigmoid/tanh)
                parameters[f'W{l}'] = np.random.randn(n_curr, n_prev) * np.sqrt(1.0 / n_prev)
            elif self.init_type == 'he':
                # He initialization (good for ReLU)
                parameters[f'W{l}'] = np.random.randn(n_curr, n_prev) * np.sqrt(2.0 / n_prev)
            else:
                raise ValueError(f"Unknown initialization type: {self.init_type}")
            
            parameters[f'b{l}'] = np.zeros((n_curr, 1))
            
        return parameters
    
    def _activate(self, Z, activation):
        """Apply activation function"""
        if activation == 'sigmoid':
            return 1 / (1 + np.exp(-np.clip(Z, -500, 500)))
        elif activation == 'tanh':
            return np.tanh(Z)
        elif activation == 'relu':
            return np.maximum(0, Z)
        else:
            raise ValueError(f"Unknown activation: {activation}")
    
    def _activate_derivative(self, Z, activation):
        """Compute derivative of activation function"""
        if activation == 'sigmoid':
            A = self._activate(Z, 'sigmoid')
            return A * (1 - A)
        elif activation == 'tanh':
            A = self._activate(Z, 'tanh')
            return 1 - A**2
        elif activation == 'relu':
            return (Z > 0).astype(float)
        else:
            raise ValueError(f"Unknown activation: {activation}")
    
    def forward_propagation(self, X):
        """Forward propagation with activation tracking"""
        self.cache = {'A0': X}
        A = X
        
        for l in range(1, self.num_layers + 1):
            W = self.parameters[f'W{l}']
            b = self.parameters[f'b{l}']
            
            Z = np.dot(W, A) + b
            
            # Choose activation for last layer (sigmoid) or hidden layers
            if l == self.num_layers:
                A = self._activate(Z, 'sigmoid')
            else:
                A = self._activate(Z, self.activation)
            
            # Store for backward propagation
            self.cache[f'Z{l}'] = Z
            self.cache[f'A{l}'] = A
            
            # Track activation statistics
            self.activation_stats[f'layer_{l}'] = {
                'mean': np.mean(A),
                'std': np.std(A),
                'min': np.min(A),
                'max': np.max(A)
            }
        
        return A
    
    def backward_propagation(self, X, Y):
        """Backward propagation with gradient tracking"""
        m = X.shape[1]
        gradients = {}
        
        # Output layer gradient
        AL = self.cache[f'A{self.num_layers}']
        dAL = -(Y / (AL + 1e-8) - (1 - Y) / (1 - AL + 1e-8))
        
        # Backward through layers
        dA = dAL
        for l in reversed(range(1, self.num_layers + 1)):
            A_prev = self.cache[f'A{l-1}']
            Z = self.cache[f'Z{l}']
            W = self.parameters[f'W{l}']
            
            # Compute gradients
            if l == self.num_layers:
                dZ = dA * self._activate_derivative(Z, 'sigmoid')
            else:
                dZ = dA * self._activate_derivative(Z, self.activation)
            
            dW = (1/m) * np.dot(dZ, A_prev.T)
            db = (1/m) * np.sum(dZ, axis=1, keepdims=True)
            
            if l > 1:
                dA = np.dot(W.T, dZ)
            
            gradients[f'dW{l}'] = dW
            gradients[f'db{l}'] = db
            
            # Track gradient norms
            self.gradient_norms[f'layer_{l}'] = {
                'dW_norm': np.linalg.norm(dW),
                'db_norm': np.linalg.norm(db),
                'dZ_norm': np.linalg.norm(dZ)
            }
        
        return gradients
    
    def compute_cost(self, AL, Y):
        """Compute binary cross-entropy cost"""
        m = Y.shape[1]
        cost = -(1/m) * np.sum(Y * np.log(AL + 1e-8) + (1 - Y) * np.log(1 - AL + 1e-8))
        return np.squeeze(cost)

print("Deep Neural Network class created successfully!")
print("Features: Gradient tracking, multiple activation functions, various initialization methods")

## Part 3: Demonstrating Vanishing Gradients

### Instructions:
1. Create a deep network with sigmoid activation (prone to vanishing gradients)
2. Observe how gradient magnitudes decrease through layers
3. Analyze the impact on learning

In [ ]:
# Generate synthetic dataset
X_data, y_data = make_classification(n_samples=1000, n_features=20, n_informative=15,
                                     n_redundant=5, n_clusters_per_class=2,
                                     random_state=42)
X_data = X_data.T
y_data = y_data.reshape(1, -1)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_data.T, y_data.T, test_size=0.2, random_state=42
)
X_train, X_test = X_train.T, X_test.T
y_train, y_test = y_train.T, y_test.T

print(f"Dataset created: {X_train.shape[1]} training samples, {X_test.shape[1]} test samples")
print(f"Input features: {X_train.shape[0]}")

In [ ]:
def visualize_gradient_flow(network, X, Y, title="Gradient Flow Analysis"):
    """Visualize gradient flow through network layers"""
    # Perform forward and backward propagation
    AL = network.forward_propagation(X)
    gradients = network.backward_propagation(X, Y)
    
    # Extract gradient norms
    layers = []
    dW_norms = []
    dZ_norms = []
    
    for l in range(1, network.num_layers + 1):
        layers.append(f"Layer {l}")
        dW_norms.append(network.gradient_norms[f'layer_{l}']['dW_norm'])
        dZ_norms.append(network.gradient_norms[f'layer_{l}']['dZ_norm'])
    
    # Create visualization
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot 1: Weight gradient norms
    axes[0, 0].bar(layers, dW_norms, color='blue', alpha=0.7)
    axes[0, 0].set_title('Weight Gradient Norms by Layer')
    axes[0, 0].set_ylabel('Gradient Norm')
    axes[0, 0].set_yscale('log')
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: Activation gradient norms
    axes[0, 1].bar(layers, dZ_norms, color='green', alpha=0.7)
    axes[0, 1].set_title('Activation Gradient Norms by Layer')
    axes[0, 1].set_ylabel('Gradient Norm')
    axes[0, 1].set_yscale('log')
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Activation statistics
    layer_nums = list(range(1, network.num_layers + 1))
    means = [network.activation_stats[f'layer_{l}']['mean'] for l in layer_nums]
    stds = [network.activation_stats[f'layer_{l}']['std'] for l in layer_nums]
    
    axes[1, 0].errorbar(layer_nums, means, yerr=stds, marker='o', capsize=5)
    axes[1, 0].set_title('Activation Statistics by Layer')
    axes[1, 0].set_xlabel('Layer Number')
    axes[1, 0].set_ylabel('Activation Mean ± Std')
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 4: Gradient flow ratio
    gradient_ratios = [dW_norms[i] / dW_norms[0] if dW_norms[0] != 0 else 0 
                      for i in range(len(dW_norms))]
    axes[1, 1].plot(layer_nums, gradient_ratios, marker='s', color='red', linewidth=2)
    axes[1, 1].set_title('Gradient Flow Ratio (Relative to First Layer)')
    axes[1, 1].set_xlabel('Layer Number')
    axes[1, 1].set_ylabel('Gradient Ratio')
    axes[1, 1].set_yscale('log')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.suptitle(title, fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    return dW_norms, dZ_norms

# Test with vanishing gradient prone network (sigmoid activation, random init)
print("Creating deep network with sigmoid activation (prone to vanishing gradients)...")
vanishing_network = DeepNeuralNetwork(
    layer_dims=[20, 50, 40, 30, 20, 10, 1],  # 6 hidden layers
    activation='sigmoid',
    init_type='random'
)

# Visualize initial gradient flow
dW_norms, dZ_norms = visualize_gradient_flow(
    vanishing_network, X_train[:, :100], y_train[:, :100],
    title="Vanishing Gradients with Sigmoid Activation"
)

print("\nGradient Analysis:")
print(f"First layer gradient norm: {dW_norms[0]:.6f}")
print(f"Last layer gradient norm: {dW_norms[-1]:.6f}")
print(f"Gradient decay ratio: {dW_norms[-1]/dW_norms[0]:.6f}")

if dW_norms[-1]/dW_norms[0] < 0.01:
    print("\n⚠️ WARNING: Severe vanishing gradient detected!")
    print("The network will have difficulty learning in early layers.")

## Part 4: Demonstrating Exploding Gradients

### Instructions:
1. Create a network configuration that leads to exploding gradients
2. Observe how gradient magnitudes increase exponentially
3. Understand the numerical instability this causes

In [ ]:
class ExplodingGradientNetwork(DeepNeuralNetwork):
    """Network configured to demonstrate exploding gradients"""
    
    def _initialize_parameters(self):
        """Initialize with large weights to cause exploding gradients"""
        parameters = {}
        
        for l in range(1, self.num_layers + 1):
            n_prev = self.layer_dims[l-1]
            n_curr = self.layer_dims[l]
            
            # Intentionally large initialization
            parameters[f'W{l}'] = np.random.randn(n_curr, n_prev) * 10
            parameters[f'b{l}'] = np.zeros((n_curr, 1))
            
        return parameters

print("Creating network prone to exploding gradients...")
exploding_network = ExplodingGradientNetwork(
    layer_dims=[20, 50, 40, 30, 20, 10, 1],
    activation='tanh',
    init_type='custom'  # Will use large weights
)

# Visualize gradient explosion
try:
    dW_norms_exp, dZ_norms_exp = visualize_gradient_flow(
        exploding_network, X_train[:, :100], y_train[:, :100],
        title="Exploding Gradients with Large Weight Initialization"
    )
    
    print("\nGradient Analysis:")
    print(f"First layer gradient norm: {dW_norms_exp[0]:.2e}")
    print(f"Last layer gradient norm: {dW_norms_exp[-1]:.2e}")
    print(f"Gradient amplification ratio: {dW_norms_exp[0]/dW_norms_exp[-1]:.2e}")
    
    if dW_norms_exp[0] > 1000:
        print("\n⚠️ WARNING: Exploding gradients detected!")
        print("The network parameters will become unstable during training.")
except:
    print("\n❌ Gradient computation failed due to numerical overflow!")
    print("This demonstrates the severe instability caused by exploding gradients.")

## Part 5: Implementing Gradient Clipping

### Instructions:
1. Implement gradient clipping to prevent exploding gradients
2. Compare different clipping strategies
3. Observe the stabilization effect on training

In [ ]:
def clip_gradients(gradients, max_norm=5.0, clip_type='norm'):
    """
    Clip gradients to prevent explosion
    
    Args:
        gradients: Dictionary of gradients
        max_norm: Maximum allowed norm
        clip_type: 'norm' for norm clipping, 'value' for value clipping
    
    Returns:
        Clipped gradients and clipping statistics
    """
    clipped_gradients = {}
    clip_stats = {'original_norm': 0, 'clipped_norm': 0, 'clip_ratio': 1.0}
    
    if clip_type == 'norm':
        # Calculate total gradient norm
        total_norm = 0
        for key in gradients:
            total_norm += np.sum(gradients[key] ** 2)
        total_norm = np.sqrt(total_norm)
        
        clip_stats['original_norm'] = total_norm
        
        # Clip if necessary
        clip_ratio = max_norm / (total_norm + 1e-8)
        clip_ratio = min(clip_ratio, 1.0)
        
        for key in gradients:
            clipped_gradients[key] = gradients[key] * clip_ratio
        
        clip_stats['clip_ratio'] = clip_ratio
        clip_stats['clipped_norm'] = min(total_norm, max_norm)
        
    elif clip_type == 'value':
        # Clip individual gradient values
        for key in gradients:
            clipped_gradients[key] = np.clip(gradients[key], -max_norm, max_norm)
            clip_stats['original_norm'] += np.sum(gradients[key] ** 2)
            clip_stats['clipped_norm'] += np.sum(clipped_gradients[key] ** 2)
        
        clip_stats['original_norm'] = np.sqrt(clip_stats['original_norm'])
        clip_stats['clipped_norm'] = np.sqrt(clip_stats['clipped_norm'])
        clip_stats['clip_ratio'] = clip_stats['clipped_norm'] / (clip_stats['original_norm'] + 1e-8)
    
    return clipped_gradients, clip_stats

# Test gradient clipping
print("Testing gradient clipping on exploding gradient network...\n")

# Reinitialize network
exploding_network = ExplodingGradientNetwork(
    layer_dims=[20, 50, 40, 30, 20, 10, 1],
    activation='tanh',
    init_type='custom'
)

# Forward and backward pass
AL = exploding_network.forward_propagation(X_train[:, :100])
gradients = exploding_network.backward_propagation(X_train[:, :100], y_train[:, :100])

# Apply different clipping strategies
print("Gradient Clipping Comparison:")
print("=" * 60)

# No clipping
original_norm = sum([np.sum(gradients[key]**2) for key in gradients])**0.5
print(f"Original gradient norm: {original_norm:.2e}")

# Norm clipping
clipped_grads_norm, stats_norm = clip_gradients(gradients, max_norm=5.0, clip_type='norm')
print(f"\nNorm clipping (max_norm=5.0):")
print(f"  Clipped norm: {stats_norm['clipped_norm']:.2f}")
print(f"  Clip ratio: {stats_norm['clip_ratio']:.4f}")

# Value clipping
clipped_grads_value, stats_value = clip_gradients(gradients, max_norm=1.0, clip_type='value')
print(f"\nValue clipping (max_value=1.0):")
print(f"  Clipped norm: {stats_value['clipped_norm']:.2f}")
print(f"  Clip ratio: {stats_value['clip_ratio']:.4f}")

print("\n✅ Gradient clipping successfully stabilizes gradient magnitudes!")

## Part 6: Batch Normalization Implementation

### Instructions:
1. Implement batch normalization to stabilize gradient flow
2. Compare network behavior with and without batch norm
3. Observe the effect on activation distributions

In [ ]:
class BatchNormNetwork(DeepNeuralNetwork):
    """Neural network with batch normalization"""
    
    def __init__(self, layer_dims, activation='relu', init_type='he', use_batch_norm=True):
        super().__init__(layer_dims, activation, init_type)
        self.use_batch_norm = use_batch_norm
        self.epsilon = 1e-8
        
        # Initialize batch norm parameters
        if use_batch_norm:
            for l in range(1, self.num_layers):
                self.parameters[f'gamma{l}'] = np.ones((layer_dims[l], 1))
                self.parameters[f'beta{l}'] = np.zeros((layer_dims[l], 1))
    
    def batch_normalize(self, Z, layer_num, mode='train'):
        """
        Apply batch normalization
        
        Args:
            Z: Pre-activation values
            layer_num: Current layer number
            mode: 'train' or 'test'
        """
        if not self.use_batch_norm or layer_num == self.num_layers:
            return Z
        
        # Compute batch statistics
        mu = np.mean(Z, axis=1, keepdims=True)
        var = np.var(Z, axis=1, keepdims=True)
        
        # Normalize
        Z_norm = (Z - mu) / np.sqrt(var + self.epsilon)
        
        # Scale and shift
        gamma = self.parameters[f'gamma{layer_num}']
        beta = self.parameters[f'beta{layer_num}']
        Z_tilde = gamma * Z_norm + beta
        
        # Store for backward pass
        self.cache[f'Z_norm{layer_num}'] = Z_norm
        self.cache[f'mu{layer_num}'] = mu
        self.cache[f'var{layer_num}'] = var
        
        return Z_tilde
    
    def forward_propagation(self, X):
        """Forward propagation with batch normalization"""
        self.cache = {'A0': X}
        A = X
        
        for l in range(1, self.num_layers + 1):
            W = self.parameters[f'W{l}']
            b = self.parameters[f'b{l}']
            
            Z = np.dot(W, A) + b
            
            # Apply batch normalization before activation
            Z = self.batch_normalize(Z, l)
            
            # Apply activation
            if l == self.num_layers:
                A = self._activate(Z, 'sigmoid')
            else:
                A = self._activate(Z, self.activation)
            
            self.cache[f'Z{l}'] = Z
            self.cache[f'A{l}'] = A
            
            # Track activation statistics
            self.activation_stats[f'layer_{l}'] = {
                'mean': np.mean(A),
                'std': np.std(A),
                'min': np.min(A),
                'max': np.max(A)
            }
        
        return A

# Compare networks with and without batch normalization
print("Creating networks for batch normalization comparison...\n")

# Network without batch norm
network_no_bn = BatchNormNetwork(
    layer_dims=[20, 50, 40, 30, 20, 10, 1],
    activation='relu',
    init_type='he',
    use_batch_norm=False
)

# Network with batch norm
network_with_bn = BatchNormNetwork(
    layer_dims=[20, 50, 40, 30, 20, 10, 1],
    activation='relu',
    init_type='he',
    use_batch_norm=True
)

# Visualize both networks
print("Network WITHOUT Batch Normalization:")
dW_no_bn, _ = visualize_gradient_flow(
    network_no_bn, X_train[:, :100], y_train[:, :100],
    title="Gradient Flow WITHOUT Batch Normalization"
)

print("\nNetwork WITH Batch Normalization:")
dW_with_bn, _ = visualize_gradient_flow(
    network_with_bn, X_train[:, :100], y_train[:, :100],
    title="Gradient Flow WITH Batch Normalization"
)

# Compare gradient stability
print("\nGradient Stability Comparison:")
print("=" * 60)
print(f"Without Batch Norm - Gradient variance: {np.var(dW_no_bn):.4f}")
print(f"With Batch Norm - Gradient variance: {np.var(dW_with_bn):.4f}")
print(f"\nImprovement factor: {np.var(dW_no_bn) / (np.var(dW_with_bn) + 1e-8):.2f}x")
print("\n✅ Batch normalization significantly stabilizes gradient flow!")

## Part 7: Training Comparison

### Instructions:
1. Train networks with different gradient stabilization techniques
2. Compare convergence speed and final performance
3. Analyze which techniques work best for your problem

In [ ]:
def train_network(network, X_train, y_train, X_test, y_test, 
                  epochs=100, learning_rate=0.01, use_clipping=False, 
                  clip_norm=5.0, verbose=True):
    """
    Train neural network with optional gradient clipping
    
    Returns:
        Training history (costs, accuracies)
    """
    train_costs = []
    test_costs = []
    train_accuracies = []
    test_accuracies = []
    gradient_norms = []
    
    for epoch in range(epochs):
        # Forward propagation
        AL_train = network.forward_propagation(X_train)
        train_cost = network.compute_cost(AL_train, y_train)
        
        # Backward propagation
        gradients = network.backward_propagation(X_train, y_train)
        
        # Track gradient norm
        grad_norm = sum([np.sum(gradients[key]**2) for key in gradients])**0.5
        gradient_norms.append(grad_norm)
        
        # Apply gradient clipping if requested
        if use_clipping:
            gradients, _ = clip_gradients(gradients, max_norm=clip_norm, clip_type='norm')
        
        # Update parameters
        for l in range(1, network.num_layers + 1):
            network.parameters[f'W{l}'] -= learning_rate * gradients[f'dW{l}']
            network.parameters[f'b{l}'] -= learning_rate * gradients[f'db{l}']
            
            # Update batch norm parameters if they exist
            if hasattr(network, 'use_batch_norm') and network.use_batch_norm and l < network.num_layers:
                if f'dgamma{l}' in gradients:
                    network.parameters[f'gamma{l}'] -= learning_rate * gradients[f'dgamma{l}']
                    network.parameters[f'beta{l}'] -= learning_rate * gradients[f'dbeta{l}']
        
        # Evaluate on test set
        AL_test = network.forward_propagation(X_test)
        test_cost = network.compute_cost(AL_test, y_test)
        
        # Calculate accuracies
        train_pred = (AL_train > 0.5).astype(float)
        test_pred = (AL_test > 0.5).astype(float)
        train_acc = np.mean(train_pred == y_train) * 100
        test_acc = np.mean(test_pred == y_test) * 100
        
        # Store history
        train_costs.append(train_cost)
        test_costs.append(test_cost)
        train_accuracies.append(train_acc)
        test_accuracies.append(test_acc)
        
        # Print progress
        if verbose and epoch % 20 == 0:
            print(f"Epoch {epoch:3d}: Train Cost={train_cost:.4f}, "
                  f"Test Cost={test_cost:.4f}, Train Acc={train_acc:.1f}%, "
                  f"Test Acc={test_acc:.1f}%, Grad Norm={grad_norm:.2e}")
    
    return {
        'train_costs': train_costs,
        'test_costs': test_costs,
        'train_accuracies': train_accuracies,
        'test_accuracies': test_accuracies,
        'gradient_norms': gradient_norms
    }

# Train different network configurations
print("Training networks with different gradient stabilization techniques...\n")
print("=" * 70)

# Configuration 1: Baseline (prone to vanishing gradients)
print("\n1. BASELINE (Sigmoid, Random Init, No Stabilization):")
baseline_network = DeepNeuralNetwork(
    layer_dims=[20, 30, 20, 10, 1],
    activation='sigmoid',
    init_type='random'
)
baseline_history = train_network(
    baseline_network, X_train, y_train, X_test, y_test,
    epochs=100, learning_rate=0.1, use_clipping=False, verbose=True
)

# Configuration 2: Better initialization
print("\n2. IMPROVED INITIALIZATION (ReLU, He Init):")
better_init_network = DeepNeuralNetwork(
    layer_dims=[20, 30, 20, 10, 1],
    activation='relu',
    init_type='he'
)
better_init_history = train_network(
    better_init_network, X_train, y_train, X_test, y_test,
    epochs=100, learning_rate=0.01, use_clipping=False, verbose=True
)

# Configuration 3: With gradient clipping
print("\n3. WITH GRADIENT CLIPPING (ReLU, He Init, Clipping):")
clipped_network = DeepNeuralNetwork(
    layer_dims=[20, 30, 20, 10, 1],
    activation='relu',
    init_type='he'
)
clipped_history = train_network(
    clipped_network, X_train, y_train, X_test, y_test,
    epochs=100, learning_rate=0.01, use_clipping=True, clip_norm=5.0, verbose=True
)

# Configuration 4: With batch normalization
print("\n4. WITH BATCH NORMALIZATION (ReLU, He Init, BatchNorm):")
bn_network = BatchNormNetwork(
    layer_dims=[20, 30, 20, 10, 1],
    activation='relu',
    init_type='he',
    use_batch_norm=True
)
bn_history = train_network(
    bn_network, X_train, y_train, X_test, y_test,
    epochs=100, learning_rate=0.01, use_clipping=False, verbose=True
)

## Part 8: Performance Comparison and Analysis

### Instructions:
1. Visualize training curves for all configurations
2. Compare convergence speed and stability
3. Draw conclusions about best practices

In [ ]:
def plot_training_comparison(histories, labels):
    """Compare training histories of different configurations"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot 1: Training cost
    for history, label in zip(histories, labels):
        axes[0, 0].plot(history['train_costs'], label=label, linewidth=2)
    axes[0, 0].set_title('Training Cost Over Time')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Cost')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Plot 2: Test accuracy
    for history, label in zip(histories, labels):
        axes[0, 1].plot(history['test_accuracies'], label=label, linewidth=2)
    axes[0, 1].set_title('Test Accuracy Over Time')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy (%)')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    
    # Plot 3: Gradient norms
    for history, label in zip(histories, labels):
        axes[1, 0].plot(history['gradient_norms'], label=label, linewidth=2, alpha=0.7)
    axes[1, 0].set_title('Gradient Norm Evolution')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Gradient Norm')
    axes[1, 0].set_yscale('log')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # Plot 4: Final comparison bar chart
    final_accuracies = [history['test_accuracies'][-1] for history in histories]
    colors = ['red', 'orange', 'green', 'blue']
    bars = axes[1, 1].bar(labels, final_accuracies, color=colors, alpha=0.7)
    axes[1, 1].set_title('Final Test Accuracy Comparison')
    axes[1, 1].set_ylabel('Accuracy (%)')
    axes[1, 1].set_ylim([0, 100])
    axes[1, 1].grid(True, alpha=0.3, axis='y')
    
    # Add value labels on bars
    for bar, acc in zip(bars, final_accuracies):
        height = bar.get_height()
        axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + 1,
                       f'{acc:.1f}%', ha='center', va='bottom')
    
    plt.suptitle('Gradient Stabilization Techniques Comparison', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Create comparison plots
histories = [baseline_history, better_init_history, clipped_history, bn_history]
labels = ['Baseline', 'Better Init', 'Gradient Clipping', 'Batch Norm']

plot_training_comparison(histories, labels)

# Performance summary
print("\nPerformance Summary:")
print("=" * 70)
print(f"{'Configuration':<20} {'Final Train Acc':<18} {'Final Test Acc':<18} {'Avg Gradient Norm'}")
print("-" * 70)

for history, label in zip(histories, labels):
    final_train_acc = history['train_accuracies'][-1]
    final_test_acc = history['test_accuracies'][-1]
    avg_grad_norm = np.mean(history['gradient_norms'])
    print(f"{label:<20} {final_train_acc:>15.1f}%  {final_test_acc:>15.1f}%  {avg_grad_norm:>18.2e}")

# Identify best configuration
best_idx = np.argmax([h['test_accuracies'][-1] for h in histories])
print(f"\n🏆 Best Configuration: {labels[best_idx]}")
print(f"   Final Test Accuracy: {histories[best_idx]['test_accuracies'][-1]:.1f}%")

## Part 9: Advanced Gradient Analysis

### Instructions:
1. Perform detailed gradient flow analysis
2. Identify problem layers in deep networks
3. Apply targeted solutions

In [ ]:
def diagnose_gradient_health(network, X, Y, threshold_vanishing=0.01, threshold_exploding=100):
    """
    Comprehensive gradient health diagnosis
    
    Returns:
        Dictionary with diagnostic information
    """
    # Forward and backward pass
    AL = network.forward_propagation(X)
    gradients = network.backward_propagation(X, Y)
    
    diagnosis = {
        'healthy_layers': [],
        'vanishing_layers': [],
        'exploding_layers': [],
        'recommendations': []
    }
    
    # Analyze each layer
    for l in range(1, network.num_layers + 1):
        grad_norm = np.linalg.norm(gradients[f'dW{l}'])
        
        if grad_norm < threshold_vanishing:
            diagnosis['vanishing_layers'].append(l)
        elif grad_norm > threshold_exploding:
            diagnosis['exploding_layers'].append(l)
        else:
            diagnosis['healthy_layers'].append(l)
    
    # Generate recommendations
    if diagnosis['vanishing_layers']:
        diagnosis['recommendations'].append(
            f"⚠️ Vanishing gradients detected in layers {diagnosis['vanishing_layers']}\n"
            "   Recommendations:\n"
            "   - Use ReLU or LeakyReLU activation instead of sigmoid/tanh\n"
            "   - Apply batch normalization\n"
            "   - Use residual connections for very deep networks\n"
            "   - Consider Xavier or He initialization"
        )
    
    if diagnosis['exploding_layers']:
        diagnosis['recommendations'].append(
            f"⚠️ Exploding gradients detected in layers {diagnosis['exploding_layers']}\n"
            "   Recommendations:\n"
            "   - Apply gradient clipping (norm or value based)\n"
            "   - Reduce learning rate\n"
            "   - Use proper weight initialization\n"
            "   - Consider L2 regularization"
        )
    
    if not diagnosis['vanishing_layers'] and not diagnosis['exploding_layers']:
        diagnosis['recommendations'].append(
            "✅ Gradient flow appears healthy!\n"
            "   Continue training with current configuration."
        )
    
    # Calculate gradient flow efficiency
    first_layer_norm = np.linalg.norm(gradients[f'dW1'])
    last_layer_norm = np.linalg.norm(gradients[f'dW{network.num_layers}'])
    
    if last_layer_norm > 0:
        flow_ratio = first_layer_norm / last_layer_norm
    else:
        flow_ratio = float('inf')
    
    diagnosis['flow_ratio'] = flow_ratio
    diagnosis['gradient_norms'] = {f'layer_{l}': np.linalg.norm(gradients[f'dW{l}']) 
                                   for l in range(1, network.num_layers + 1)}
    
    return diagnosis

# Test gradient health diagnosis on different networks
print("Performing Gradient Health Diagnosis...\n")
print("=" * 70)

test_configs = [
    ('Sigmoid Network (Deep)', [20, 40, 30, 20, 10, 5, 1], 'sigmoid', 'random'),
    ('ReLU Network (Optimized)', [20, 40, 30, 20, 10, 5, 1], 'relu', 'he'),
    ('Tanh Network (Xavier)', [20, 40, 30, 20, 10, 5, 1], 'tanh', 'xavier')
]

for name, layers, activation, init in test_configs:
    print(f"\n{name}:")
    print("-" * 40)
    
    test_network = DeepNeuralNetwork(layers, activation, init)
    diagnosis = diagnose_gradient_health(test_network, X_train[:, :100], y_train[:, :100])
    
    print(f"Healthy layers: {diagnosis['healthy_layers']}")
    print(f"Vanishing gradient layers: {diagnosis['vanishing_layers']}")
    print(f"Exploding gradient layers: {diagnosis['exploding_layers']}")
    print(f"Gradient flow ratio (first/last): {diagnosis['flow_ratio']:.4f}")
    
    for rec in diagnosis['recommendations']:
        print(f"\n{rec}")

## Part 10: Best Practices Summary

### Instructions:
1. Review the summary of techniques learned
2. Understand when to apply each technique
3. Complete the exercises to reinforce learning

In [ ]:
# Create a reference implementation with all best practices
class OptimizedDeepNetwork:
    """Production-ready deep network with gradient stabilization"""
    
    def __init__(self, layer_dims, config=None):
        """
        Initialize optimized deep network
        
        Args:
            layer_dims: List of layer dimensions
            config: Dictionary with configuration options
        """
        self.layer_dims = layer_dims
        self.num_layers = len(layer_dims) - 1
        
        # Default configuration
        default_config = {
            'activation': 'relu',
            'output_activation': 'sigmoid',
            'initialization': 'he',
            'use_batch_norm': True,
            'use_dropout': False,
            'dropout_rate': 0.2,
            'gradient_clipping': True,
            'clip_norm': 5.0,
            'learning_rate': 0.001,
            'learning_rate_decay': 0.99,
            'regularization': 'l2',
            'reg_lambda': 0.01
        }
        
        self.config = {**default_config, **(config or {})}
        
        print("Optimized Deep Network Configuration:")
        print("=" * 50)
        for key, value in self.config.items():
            print(f"{key:<20}: {value}")
        print("=" * 50)
        print(f"\nNetwork Architecture: {layer_dims}")
        print(f"Total Parameters: {self._count_parameters():,}")
    
    def _count_parameters(self):
        """Count total number of parameters"""
        total = 0
        for l in range(1, self.num_layers + 1):
            total += self.layer_dims[l] * self.layer_dims[l-1]  # Weights
            total += self.layer_dims[l]  # Biases
            if self.config['use_batch_norm'] and l < self.num_layers:
                total += 2 * self.layer_dims[l]  # Gamma and beta
        return total

# Create optimized network
print("\n" + "="*70)
print("CREATING PRODUCTION-READY OPTIMIZED NETWORK")
print("="*70 + "\n")

optimized_network = OptimizedDeepNetwork(
    layer_dims=[20, 64, 32, 16, 8, 1],
    config={
        'activation': 'relu',
        'initialization': 'he',
        'use_batch_norm': True,
        'gradient_clipping': True,
        'clip_norm': 5.0,
        'learning_rate': 0.001,
        'regularization': 'l2',
        'reg_lambda': 0.01
    }
)

## Exercise Solutions and Key Takeaways

### Key Techniques for Gradient Stabilization:

1. **Proper Weight Initialization**
   - Xavier/Glorot: Best for sigmoid/tanh activations
   - He: Best for ReLU and variants
   - Avoid random initialization with very small or large values

2. **Activation Function Selection**
   - ReLU: Reduces vanishing gradients, but can cause dead neurons
   - LeakyReLU/ELU: Addresses dead neuron problem
   - Avoid sigmoid/tanh in hidden layers of deep networks

3. **Gradient Clipping**
   - Norm clipping: Preserves gradient direction
   - Value clipping: Simple but can change gradient direction
   - Essential for RNNs and very deep networks

4. **Batch Normalization**
   - Normalizes inputs to each layer
   - Allows higher learning rates
   - Reduces sensitivity to initialization

5. **Architecture Considerations**
   - Residual connections for very deep networks
   - Skip connections to preserve gradient flow
   - Careful layer width design

### Troubleshooting Guide:

| Problem | Symptoms | Solutions |
|---------|----------|----------|
| Vanishing Gradients | Slow/no learning in early layers, gradients → 0 | Use ReLU, batch norm, residual connections |
| Exploding Gradients | NaN values, unstable training, gradients → ∞ | Gradient clipping, reduce learning rate, proper init |
| Dead Neurons | Zero activations, no learning | LeakyReLU, careful learning rate, batch norm |
| Slow Convergence | High loss persists, poor accuracy | Better initialization, normalize inputs, adjust architecture |

## Lab Complete! 🎉

### What You've Learned:
✅ Identified and diagnosed vanishing/exploding gradient problems  
✅ Implemented gradient clipping techniques  
✅ Applied batch normalization for stability  
✅ Compared different initialization strategies  
✅ Built production-ready gradient stabilization solutions  

### Next Steps:
1. Experiment with different network depths and observe gradient behavior
2. Try implementing Layer Normalization as an alternative to Batch Norm
3. Explore gradient accumulation for large batch training
4. Investigate adaptive learning rate methods (Adam, RMSprop)

### Additional Challenges:
1. Implement gradient checkpointing for memory-efficient training
2. Create a custom gradient clipping strategy based on layer depth
3. Build an automatic gradient health monitoring system
4. Compare different normalization techniques (Group Norm, Instance Norm)

### Cleanup:
```python
# Clear variables to free memory
import gc
gc.collect()
```